In [3]:
!pip install -r requirements.txt
!python -m pip install --upgrade pip

#### Initialization

In [4]:
## libraries 
from sqlalchemy import create_engine, text
from pymongo import MongoClient
import mysql.connector
from time import time 
import pandas as pd 
import dotenv
import json 
import os 

In [13]:
## database initialization
dotenv.load_dotenv()


MYSQL_CONFIG = {
    'username': os.environ['mysql_username'],
    'password': os.environ['mysql_password'],
    'host'    : os.environ['mysql_hostname'],
}

MONGO_DB_CONDIg = {
    'username': os.environ['mongo_username'],
    'password': os.environ['mongo_password'],
    'appName' : os.environ['mongo_hostname'],
    'port'    : os.environ['mongo_port']
}

DB_NAME = 'project_test'

In [11]:
def reset_sqlite_database(engine, dbname : str):
    with engine.connect() as connection:
        connection.execute(text(f"DROP DATABASE IF EXISTS {dbname}"))
        connection.execute(text(f"CREATE DATABASE {dbname}"))
    print(f'Recreated the database : [{dbname}]')

In [14]:

db_url = f"mysql+mysqlconnector://{MYSQL_CONFIG['username']}:{MYSQL_CONFIG['password']}@{MYSQL_CONFIG['host']}:3306"
engine = create_engine(db_url)
reset_sqlite_database(engine, dbname = DB_NAME)
db_url = f"mysql+mysqlconnector://{MYSQL_CONFIG['username']}:{MYSQL_CONFIG['password']}@{MYSQL_CONFIG['host']}:3306/{DB_NAME}"
engine = create_engine(db_url)

Recreated the database : [project_test]


In [22]:
transaction_filename = os.path.join('finance_transaction', 'transactions_data.csv')
cards_filename = os.path.join('finance_transaction', 'cards_data.csv')
users_filename = os.path.join('finance_transaction', 'users_data.csv')

n_rows = 500 # using limited rows due to large number of rows  
transaction_df = pd.read_csv(transaction_filename, nrows= n_rows)
cards_df = pd.read_csv(cards_filename, nrows= n_rows)
users_df = pd.read_csv(users_filename, nrows= n_rows)

In [23]:
transaction_df.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [24]:
cards_df.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [25]:
users_df.head()

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


In [2]:
client = MongoClient("mongodb://localhost:27017")

In [ ]:
db = client['group_25']
print(client.list_database_names())

['JD_student', 'admin', 'config', 'local', 'mydatabase']
